
![Image Description](/images/Operations_Research_Methodology_Diagram.png)







I've been working on a workforce scheduling problem using linear programming in Python with the PuLP library. I'm trying to assign employees to different brands, ensuring certain conditions are met. Here's a detailed explanation of the problem and the code I've implemented:

Problem Statement:
Assign a fixed number of employees to different brands.
Ensure that an employee works on only one brand per day.
An employee can work a maximum of 9 hours and a minimum of 5 hours consecutively.
Need to fulfill staffing requirements for each brand.

In [7]:
import itertools

from pulp import LpProblem, LpVariable, LpBinary, lpSum

n_weeks = 2
days_per_week = 7
total_days = n_weeks * days_per_week
shop_open = 10  # 10 AM
shop_close = 20  # 8 PM
min_working_hours = 5
max_working_hours = 9
n_employees = 25

### DATA

# 5-day or 4-day contract type (60% each for now)
five_days_count = n_employees * 0.6

brands = {
    "PHONE": ["PHONE1", "PHONE2"],
    "TV": ["TV1", "TV2", "TV3"]
}
brand_items = tuple(itertools.chain(*brands.values()))

# employee table  e# : ( 4/5 day , [brand_item, ...] )
employees = {f'e{i}': (5*n_weeks if i < five_days_count else 4*n_weeks, [brand_items[i % len(brand_items)]]) for i in range(n_employees)}
# print(employees)

staffing_requirements = {
    "PHONE1": 1,
    "PHONE2": 1,
    "TV1": 1,
    "TV2": 1,
    "TV3": 1}

# the hours on which it is possible to commence a 5-hr shift
poss_starts = [hr for hr in range(shop_open, shop_close - min_working_hours + 1)]

prob = LpProblem('shift_sked')

### SETS / INDEXES

hours = list(range(shop_open, shop_close))  # convenience for readability...
days = list(range(total_days))
#                                                     Domain trim-down:  vvvv                 vvvv
EDHB_employees = {(e, d, h, b) for e in employees for d in days for h in poss_starts for b in employees[e][1]}
EDHB_brand_items = {(e, d, h, b) for e in employees for d in days for h in hours for b in employees[e][1]}

### VARS

start_shift = LpVariable.dicts('start', indices=EDHB_employees, cat=LpBinary) # e starts shift on day d, hour h, for brand item b
covers = LpVariable.dicts('covers', indices=EDHB_brand_items, cat=LpBinary)  # e covers brand item b on day d, hour h
max_shifts = LpVariable('max_shifts')  # the max shifts by any employee

### OBJ:  Minimize shift starts
prob += lpSum(start_shift)

# alternate objectives for experimenting...
# prob += lpSum(covers)  # should produce days*hours*requirements
# prob += max_shifts  # the max number of shifts by any employee  NOTE:  This is tougher solve, longer...

### CONSTRAINTS

# 1 & 2.  Limit shift starts by 4/5 day limit, and can only start 1 shift/day
for e in employees:
    prob += sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1]) <= employees[e][0]
    for d in days:
        prob += sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1]) <= 1

# 3. Link coverage to shift starting
for (e, d, h, b) in EDHB_brand_items:
    if b not in employees[e][1]:  # this employee cannot 'cover' this item
        prob += covers[e, d, h, b] <= 0
    elif h in poss_starts:  # a start or continued coverage can work...
        prev_hour = covers[e, d, h-1, b] if h > shop_open else None
        prob += covers[e, d, h, b] <= start_shift[e, d, h, b] + prev_hour
    else:  # only previous hour coverage can work (start not possible)
        prob += covers[e, d, h, b] <= covers[e, d, h-1, b]

# 4.  min/max coverage
for e in employees:
    for d in days:
        starts_shift = sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1])
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) <= max_working_hours * starts_shift
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) >= min_working_hours * starts_shift

# 5.  brand-item coverage minimums
for (d, h, b) in itertools.product(days, hours, brand_items):
    prob += sum(covers[e, d, h, b] for e in employees if b in employees[e][1]) >= staffing_requirements[b]

# 6.  Capture max shifts
for e in employees:
    prob += max_shifts >= sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1])

# print(prob)

cbc_path = '/opt/homebrew/opt/cbc/bin/cbc'
solver = pulp.COIN_CMD(path=cbc_path)
res = prob.solve(solver)

# highs_path = '/opt/homebrew/bin/highs'
# solver_2 = pulp.HiGHS_CMD(path=highs_path)
# res = prob.solve(solver_2)

NameError: name 'pulp' is not defined

In [9]:
import pandas as pd
from datetime import datetime, timedelta, time
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value, PULP_CBC_CMD

file_path = r'C:\Users\Alvaro\Documents\Facultad\MBZUAI\Internship\Etihad\Internship\Zonal Allocation\Automated allocations\Automated Allocation Excel v6 - corrected dates.xlsm'
# Load data
manpower_df = pd.read_excel(file_path, sheet_name='Manpower').head(30)
flights_df = pd.read_excel(file_path, sheet_name='Flights').head(30)


# Let's handle the NaN values
manpower_df['Main Certifications'] = manpower_df['Main Certifications'].apply(lambda x: [] if pd.isna(x) else x)
manpower_df['Cat-A Certifications'] = manpower_df['Cat-A Certifications'].apply(lambda x: [] if pd.isna(x) else x)


In [10]:
# Extract certifiers, aircraft, bays, and shifts from the data
certifiers = manpower_df['Name'].unique()
aircraft_wp = flights_df['WP'].unique()
bays = flights_df['Bay'].unique()

# Create dictionaries for certifier and aircraft information
certifier_info = manpower_df.set_index('Name').to_dict('index')
aircraft_info = flights_df.set_index('WP').to_dict('index')

# Define what a long stop is, in minutes 
long_stop = 5*60

In [63]:
########################### COSTS ########################################

# Function to calculate movement cost (or time) between bays
# If they are of the same area (for instance, both are 600), I define the movement cost as 2 minutes per bay, with a maximum of 10 minutes
# If they are of different areas, I set the movement cost as 30 minutes
def calculate_movement_cost(bay1, bay2):
    if bay1 // 100 == bay2 // 100:
        return min(2 * abs(bay1 - bay2), 10)
    else:
        return 30

# Create a movement time matrix. This matrix will be for all the possible pairs of aircraft
movement_cost_matrix = {}
for a1 in aircraft_wp:
    for a2 in aircraft_wp:
        movement_cost_matrix[(a1, a2)] = calculate_movement_cost(aircraft_info[a1]['Bay'], aircraft_info[a2]['Bay'])

# We can also define a cost of changing bay zones.
def calculate_zone_change_cost(bay_zone_1, bay_zone_2):
    if bay_zone_1 != bay_zone_2:
        return 100
    else:
        return 0

# Costs of assigning Main Certifier, Cat-A Certifier and someone with no certifications
# The No Certifications needs to be done in order to always get a solution
def calculate_certification_cost(certifier, aircraft):
    aircraft_type = aircraft_info[aircraft]["Aircraft type"]
    main_certifications = certifier_info[certifier]["Main Certifications"]
    cat_a_certifications = certifier_info[certifier]["Cat-A Certifications"]
    if aircraft_type in main_certifications:
        return 0
    elif aircraft_type in cat_a_certifications:
        return 50
    else:
        return 100000  # Large penalty to discourage assignment


# Cost of assigning a shift leader
def calculate_shift_leader_cost(certifier):
    if certifier_info[certifier]['Type'] == 'LE':
        return 100
    else:
        return 0

# Cost of assigning a quality control certifier
def calculate_quality_control_cost(certifier):
    if certifier_info[certifier]['Primary Bay Zone'] == 'Quality Control':
        return 200
    else:
        return 0


# Define the cost for assigning multiple aircraft to the same certifier. It should not grow linearly, but more exponentially
# 1 aicraft -> 50, 2 aircraft -> 150, 3 aircraft -> 300, 4 aircraft -> 500, 5 aircraft -> 750, 6 aircraft -> 1050
def calculate_assignment_cost(num_assigned_aircraft):
    if num_assigned_aircraft == 0:
        return 0
    if num_assigned_aircraft == 1:
        return 50
    elif num_assigned_aircraft == 2:
        return 150
    elif num_assigned_aircraft == 3:
        return 300
    elif num_assigned_aircraft == 4:
        return 500
    elif num_assigned_aircraft == 5:
        return 750
    elif num_assigned_aircraft == 6:
        return 1050
    else:
        return 1400

    # Define the cost of assigning aicraft with same departure time

In [66]:
# Initialize the problem
problem = LpProblem("Aircraft_Assignment", LpMinimize)
total_cost = 0

# Define decision variables
assignment = LpVariable.dicts("Assign", (certifiers, aircraft_wp), 0, 1, cat="Binary")

# Auxiliary variables for movement
movement = LpVariable.dicts("Movement", (certifiers, aircraft_wp, aircraft_wp), 0, 1, cat='Binary')

# Calculate total costs
total_cost = lpSum(
    [
        movement_cost_matrix[(a1, a2)] * movement[c][a1][a2]
        for c in certifiers
        for a1 in aircraft_wp
        for a2 in aircraft_wp
    ]
)

# Calculate assignment costs and add them to the total cost
# Define binary variables for assignment cost function
assignment_costs = {
    c: LpVariable.dicts(f"AssignmentCost_{c}", range(7), 0, 1, cat="Binary")
    for c in certifiers
}

for c in certifiers:
    # Ensure that exactly one cost level is selected
    problem += lpSum([assignment_costs[c][i] for i in range(7)]) == 1

    # Link assignment cost levels to the number of assigned aircraft
    problem += lpSum([assignment[c][a] for a in aircraft_wp]) == lpSum(
        [i * assignment_costs[c][i] for i in range(7)]
    )

    # Calculate assignment cost based on the number of assigned aircraft
    total_cost += lpSum(
        [calculate_assignment_cost(i) * assignment_costs[c][i] for i in range(7)]
    )
    

# Add the additional cost components to the total cost
total_cost += lpSum(
    [
        assignment[c][a]
        * (
            calculate_certification_cost(c, a)
            + calculate_shift_leader_cost(c)
            + calculate_quality_control_cost(c)
        )
        for c in certifiers
        for a in aircraft_wp
    ]
)

# Add the total cost to the problem
problem += total_cost, "Minimize_Total_Cost"

# Constraints
# Each aircraft should be assigned to exactly one certifier
for a in aircraft_wp:
    problem += lpSum([assignment[c][a] for c in certifiers]) == 1

# Certifiers should be assigned to aircraft they have certification, as Main Certification or Cat-A certification. If they don't have it, they can't be assigned
for c in certifiers:
    for a in aircraft_wp:
        if (
            aircraft_info[a]["Aircraft type"]
            not in certifier_info[c]["Main Certifications"]
            and aircraft_info[a]["Aircraft type"]
            not in certifier_info[c]["Cat-A Certifications"]
        ):
            problem += assignment[c][a] == 0

# Add the constraint that certifiers should be assigned to aircraft in their shift range + 1 hour
for c in certifiers:
    for a in aircraft_wp:
        arrival_time = aircraft_info[a]['Arrival time']
        departure_time = aircraft_info[a]['Departure time']

        shift_start_buffered = certifier_info[c]['Shift Start'] - timedelta(hours=1)
        shift_end_buffered = certifier_info[c]['Shift End'] + timedelta(hours=1)

        # Check if the aircraft's arrival or departure time falls within the buffered shift range
        if not (shift_start_buffered <= arrival_time <= shift_end_buffered or
                shift_start_buffered <= departure_time <= shift_end_buffered):
            problem += assignment[c][a] == 0

# Certifiers should be assigned to a maximum of 2 aircraft with long layovers
for c in certifiers:
    problem += lpSum([assignment[c][a] for a in aircraft_wp if aircraft_info[a]['Ground Time (minutes)'] > long_stop]) <= 2

# Link movement variables with assignments
for c in certifiers:
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                problem += movement[c][a1][a2] >= assignment[c][a1] + assignment[c][a2] - 1
                problem += movement[c][a1][a2] <= assignment[c][a1]
                problem += movement[c][a1][a2] <= assignment[c][a2]

# Solve the problem
solver = PULP_CBC_CMD(msg=1, timeLimit=120, presolve="on", strong=2, gapRel=0.05)
problem.solve(solver)

# Check the solution
print(f"Status: {LpStatus[problem.status]}")
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

Status: Optimal
Total cost: 2968.0


In [67]:
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

# Print the costs and assignments
no_certification_assignments = []

for c in certifiers:
    num_aircraft_assigned = sum(assignment[c][a].varValue for a in aircraft_wp)
    assignment_cost = calculate_assignment_cost(num_aircraft_assigned)
    print(f"Certifier {c}:")
    print(f"  Number of aircraft assigned: {num_aircraft_assigned}")
    print(f"  Assignment cost: {assignment_cost}")

    movement_cost = 0
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                movement_cost += (
                    movement_cost_matrix[(a1, a2)]
                    * assignment[c][a1].varValue
                    * assignment[c][a2].varValue
                )
    print(f"  Movement cost: {movement_cost}")

    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            certification_cost = calculate_certification_cost(c, a)
            print(f"  Aircraft {a}:")
            print(f"    Certification cost: {certification_cost}")
            if certification_cost == 100000:
                no_certification_assignments.append((c, a))

# Print assignments without certifications
if no_certification_assignments:
    print("Assignments without proper certifications:")
    for certifier, aircraft in no_certification_assignments:
        print(
            f"  Certifier {certifier} is assigned to Aircraft {aircraft} without proper certification"
        )

# Print total cost
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

# Print final assignments
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Certifier A.OBAID is assigned to Aircraft EIX/L-150724-3
Certifier A.SHUAIBI is assigned to Aircraft EIA/L-150724-3
Certifier SANDEEP is assigned to Aircraft AEJ/L-150724-2
Certifier AMIN S is assigned to Aircraft AEF/L-150724-2
Certifier AMIN S is assigned to Aircraft EII/L-150724-4
Certifier ZAHID is assigned to Aircraft EIW/L-150724-2
Certifier ZAHID is assigned to Aircraft EIN/L-150724-4
Certifier ADIL is assigned to Aircraft EIH/L-150724-3
Certifier ADIL is assigned to Aircraft EIV/L-150724-2
Certifier MITSIS is assigned to Aircraft AEB/L-150724-2
Certifier MITSIS is assigned to Aircraft EIS/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EJA/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft AEE/L-150724-2
Certifier OSMAN is assigned to Aircraft EIU/L-150724-3
Certifier OSMAN is assigned to Aircraft EJA/L-150724-3
Certifier JAHANGIR is assigned to Aircraft EII/L-150724-3
Certifier JAHANGIR is assigned to Aircraft EIT/L-150724-2
Certifier MARK C. is assigned to Airc

In [15]:
# Check the solution status
status = LpStatus[problem.status]
print(f"Status: {status}")

if status == "Infeasible":
    print("The problem is infeasible. Analyzing constraints...")

    # Iterate over constraints to find those that are not satisfied
    for name, constraint in problem.constraints.items():
        if constraint.value() > constraint.constant:
            print(f"Constraint {name} is not satisfied.")
            print(f" - Expression: {constraint}")
            print(f" - Left-hand side value: {constraint.value()}")
            print(f" - Right-hand side value: {constraint.constant}")


# Check the solution (if feasible or optimal)
if status in ["Optimal", "Feasible"]:
    for c in certifiers:
        for a in aircraft_wp:
            if assignment[c][a].varValue == 1:
                print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Optimal
Certifier A.OBAID is assigned to Aircraft AEC/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft EIX/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft AEI/L-150724-2
Certifier SANDEEP is assigned to Aircraft EIH/L-150724-3
Certifier AMIN S is assigned to Aircraft EJA/L-150724-2
Certifier AMIN S is assigned to Aircraft AEE/L-150724-2
Certifier ZAHID is assigned to Aircraft EIW/L-150724-2
Certifier ZAHID is assigned to Aircraft EIY/L-150724-2
Certifier ADIL is assigned to Aircraft EIT/L-150724-2
Certifier ADIL is assigned to Aircraft EJA/L-150724-3
Certifier MITSIS is assigned to Aircraft AEJ/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft AEF/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EII/L-150724-4
Certifier OSMAN is assigned to Aircraft EIN/L-150724-3
Certifier OSMAN is assigned to Aircraft EIU/L-150724-3
Certifier OSMAN is assigned to Aircraft AEG/L-1G0724-2
Certifier JAHANGIR is assigned to Aircraft EIV/L-150724-2
Certifier MARK C. is 

In [69]:
import pandas as pd

# Prepare data for the Excel file
data = []

for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            data.append({"Certifier": c, "Aircraft": a, "Assignment": 1})

# Collect cost and assignment details
cost_data = []
no_certification_assignments = []

for c in certifiers:
    num_aircraft_assigned = sum(assignment[c][a].varValue for a in aircraft_wp)
    assignment_cost = calculate_assignment_cost(num_aircraft_assigned)
    movement_cost = 0
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                movement_cost += (
                    movement_cost_matrix[(a1, a2)]
                    * assignment[c][a1].varValue
                    * assignment[c][a2].varValue
                )
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            certification_cost = calculate_certification_cost(c, a)
            if certification_cost == 100000:
                no_certification_assignments.append((c, a))
            cost_data.append(
                {
                    "Certifier": c,
                    "Aircraft": a,
                    "Number of Aircraft Assigned": num_aircraft_assigned,
                    "Assignment Cost": assignment_cost,
                    "Movement Cost": movement_cost,
                    "Certification Cost": certification_cost,
                }
            )

# Total cost
total_cost_value = value(problem.objective)

# Assignments without certifications
no_certification_data = [
    {
        "Certifier": certifier,
        "Aircraft": aircraft,
        "Issue": "Assigned without proper certification",
    }
    for certifier, aircraft in no_certification_assignments
]

# Write to Excel file
with pd.ExcelWriter("assignments_and_costs.xlsx") as writer:
    pd.DataFrame(data).to_excel(writer, sheet_name="Assignments", index=False)
    pd.DataFrame(cost_data).to_excel(writer, sheet_name="Cost Details", index=False)
    pd.DataFrame(no_certification_data).to_excel(
        writer, sheet_name="Issues", index=False
    )
    pd.DataFrame([{"Total Cost": total_cost_value}]).to_excel(
        writer, sheet_name="Total Cost", index=False
    )

In [75]:
import pandas as pd
from datetime import datetime, timedelta, time
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value, PULP_CBC_CMD

file_path = r"C:\Users\Alvaro\Documents\Facultad\MBZUAI\Internship\Etihad\Internship\Zonal Allocation\Automated allocations\Automated Allocation Excel v6 - corrected dates.xlsm"
# Load data
manpower_df = pd.read_excel(file_path, sheet_name="Manpower").head(30)
flights_df = pd.read_excel(file_path, sheet_name="Flights").head(30)

# Let's handle the NaN values
manpower_df["Main Certifications"] = manpower_df["Main Certifications"].apply(
    lambda x: [] if pd.isna(x) else x
)
manpower_df["Cat-A Certifications"] = manpower_df["Cat-A Certifications"].apply(
    lambda x: [] if pd.isna(x) else x
)
# Extract certifiers, aircraft, bays, and shifts from the data
certifiers = manpower_df["Name"].unique()
aircraft_wp = flights_df["WP"].unique()
bays = flights_df["Bay"].unique()

# Create dictionaries for certifier and aircraft information
certifier_info = manpower_df.set_index("Name").to_dict("index")
aircraft_info = flights_df.set_index("WP").to_dict("index")

# Define what a long stop is, in minutes
long_stop = 5 * 60


def calculate_movement_cost(bay1, bay2):
    if bay1 // 100 == bay2 // 100:
        return min(2 * abs(bay1 - bay2), 10)
    else:
        return 30


# Create a movement time matrix. This matrix will be for all the possible pairs of aircraft
movement_cost_matrix = {}
for a1 in aircraft_wp:
    for a2 in aircraft_wp:
        movement_cost_matrix[(a1, a2)] = calculate_movement_cost(
            aircraft_info[a1]["Bay"], aircraft_info[a2]["Bay"]
        )


# We can also define a cost of changing bay zones.
def calculate_zone_change_cost(bay_zone_1, bay_zone_2):
    if bay_zone_1 != bay_zone_2:
        return 100
    else:
        return 0


# Costs of assigning Main Certifier, Cat-A Certifier and someone with no certifications
# The No Certifications needs to be done in order to always get a solution
def calculate_certification_cost(certifier, aircraft):
    aircraft_type = aircraft_info[aircraft]["Aircraft type"]
    main_certifications = certifier_info[certifier]["Main Certifications"]
    cat_a_certifications = certifier_info[certifier]["Cat-A Certifications"]
    if aircraft_type in main_certifications:
        return 0
    elif aircraft_type in cat_a_certifications:
        return 50
    else:
        return 100000  # Large penalty to discourage assignment


# Cost of assigning a shift leader
def calculate_shift_leader_cost(certifier):
    if certifier_info[certifier]["Type"] == "LE":
        return 100
    else:
        return 0


# Cost of assigning a quality control certifier
def calculate_quality_control_cost(certifier):
    if certifier_info[certifier]["Primary Bay Zone"] == "Quality Control":
        return 200
    else:
        return 0


# Define the cost for assigning multiple aircraft to the same certifier. It should not grow linearly, but more exponentially
# 1 aicraft -> 50, 2 aircraft -> 150, 3 aircraft -> 300, 4 aircraft -> 500, 5 aircraft -> 750, 6 aircraft -> 1050
def calculate_assignment_cost(num_assigned_aircraft):
    if num_assigned_aircraft == 0:
        return 0
    if num_assigned_aircraft == 1:
        return 50
    elif num_assigned_aircraft == 2:
        return 150
    elif num_assigned_aircraft == 3:
        return 300
    elif num_assigned_aircraft == 4:
        return 500
    elif num_assigned_aircraft == 5:
        return 750
    elif num_assigned_aircraft == 6:
        return 1050
    else:
        return 1400


# Initialize the problem
problem = LpProblem("Aircraft_Assignment", LpMinimize)
total_cost = 0

# Define decision variables
assignment = LpVariable.dicts("Assign", (certifiers, aircraft_wp), 0, 1, cat="Binary")

# Auxiliary variables for movement
movement = LpVariable.dicts(
    "Movement", (certifiers, aircraft_wp, aircraft_wp), 0, 1, cat="Binary"
)

# Calculate total costs
total_cost = lpSum(
    [
        movement_cost_matrix[(a1, a2)] * movement[c][a1][a2]
        for c in certifiers
        for a1 in aircraft_wp
        for a2 in aircraft_wp
    ]
)

# Calculate assignment costs and add them to the total cost
# Define binary variables for assignment cost function
assignment_costs = {
    c: LpVariable.dicts(f"AssignmentCost_{c}", range(7), 0, 1, cat="Binary")
    for c in certifiers
}

for c in certifiers:
    # Ensure that exactly one cost level is selected
    problem += lpSum([assignment_costs[c][i] for i in range(7)]) == 1

    # Link assignment cost levels to the number of assigned aircraft
    problem += lpSum([assignment[c][a] for a in aircraft_wp]) == lpSum(
        [i * assignment_costs[c][i] for i in range(7)]
    )

    # Calculate assignment cost based on the number of assigned aircraft
    total_cost += lpSum(
        [calculate_assignment_cost(i) * assignment_costs[c][i] for i in range(7)]
    )

# Add the additional cost components to the total cost
total_cost += lpSum(
    [
        assignment[c][a]
        * (
            calculate_certification_cost(c, a)
            + calculate_shift_leader_cost(c)
            + calculate_quality_control_cost(c)
        )
        for c in certifiers
        for a in aircraft_wp
    ]
)

# Add the total cost to the problem
problem += total_cost, "Minimize_Total_Cost"

# Constraints
# Each aircraft should be assigned to exactly one certifier
for a in aircraft_wp:
    problem += lpSum([assignment[c][a] for c in certifiers]) == 1

# Certifiers should be assigned to aircraft they have certification, as Main Certification or Cat-A certification. If they don't have it, they can't be assigned
for c in certifiers:
    for a in aircraft_wp:
        if (
            aircraft_info[a]["Aircraft type"]
            not in certifier_info[c]["Main Certifications"]
            and aircraft_info[a]["Aircraft type"]
            not in certifier_info[c]["Cat-A Certifications"]
        ):
            problem += assignment[c][a] == 0

# Add the constraint that certifiers should be assigned to aircraft in their shift range + 1 hour
for c in certifiers:
    for a in aircraft_wp:
        arrival_time = aircraft_info[a]["Arrival time"]
        departure_time = aircraft_info[a]["Departure time"]

        shift_start_buffered = certifier_info[c]["Shift Start"] - timedelta(hours=1)
        shift_end_buffered = certifier_info[c]["Shift End"] + timedelta(hours=1)

        # Check if the aircraft's arrival or departure time falls within the buffered shift range
        if not (
            shift_start_buffered <= arrival_time <= shift_end_buffered
            or shift_start_buffered <= departure_time <= shift_end_buffered
        ):
            problem += assignment[c][a] == 0

# Certifiers should be assigned to a maximum of 2 aircraft with long layovers
for c in certifiers:
    problem += (
        lpSum(
            [
                assignment[c][a]
                for a in aircraft_wp
                if aircraft_info[a]["Ground Time (minutes)"] > long_stop
            ]
        )
        <= 2
    )

# Link movement variables with assignments
for c in certifiers:
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                problem += (
                    movement[c][a1][a2] >= assignment[c][a1] + assignment[c][a2] - 1
                )
                problem += movement[c][a1][a2] <= assignment[c][a1]
                problem += movement[c][a1][a2] <= assignment[c][a2]

# Solve the problem
solver = PULP_CBC_CMD(msg=1, timeLimit=120, presolve="on", strong=2, gapRel=0.05)
problem.solve(solver)

# Check the solution
print(f"Status: {LpStatus[problem.status]}")
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

# Print the costs and assignments
no_certification_assignments = []

for c in certifiers:
    num_aircraft_assigned = sum(assignment[c][a].varValue for a in aircraft_wp)
    assignment_cost = calculate_assignment_cost(num_aircraft_assigned)
    print(f"Certifier {c}:")
    print(f"  Number of aircraft assigned: {num_aircraft_assigned}")
    print(f"  Assignment cost: {assignment_cost}")

    movement_cost = 0
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                movement_cost += (
                    movement_cost_matrix[(a1, a2)]
                    * assignment[c][a1].varValue
                    * assignment[c][a2].varValue
                )
    print(f"  Movement cost: {movement_cost}")

    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            certification_cost = calculate_certification_cost(c, a)
            print(f"  Aircraft {a}:")
            print(f"    Certification cost: {certification_cost}")
            if certification_cost == 100000:
                no_certification_assignments.append((c, a))

# Print assignments without certifications
if no_certification_assignments:
    print("Assignments without proper certifications:")
    for certifier, aircraft in no_certification_assignments:
        print(
            f"  Certifier {certifier} is assigned to Aircraft {aircraft} without proper certification"
        )

# Print total cost
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

# Print final assignments
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Optimal
Total cost: 2968.0
Certifier A.OBAID is assigned to Aircraft EIX/L-150724-3
Certifier A.SHUAIBI is assigned to Aircraft EIA/L-150724-3
Certifier SANDEEP is assigned to Aircraft AEJ/L-150724-2
Certifier AMIN S is assigned to Aircraft AEF/L-150724-2
Certifier AMIN S is assigned to Aircraft EII/L-150724-4
Certifier ZAHID is assigned to Aircraft EIW/L-150724-2
Certifier ZAHID is assigned to Aircraft EIN/L-150724-4
Certifier ADIL is assigned to Aircraft EIH/L-150724-3
Certifier ADIL is assigned to Aircraft EIV/L-150724-2
Certifier MITSIS is assigned to Aircraft AEB/L-150724-2
Certifier MITSIS is assigned to Aircraft EIS/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EJA/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft AEE/L-150724-2
Certifier OSMAN is assigned to Aircraft EIU/L-150724-3
Certifier OSMAN is assigned to Aircraft EJA/L-150724-3
Certifier JAHANGIR is assigned to Aircraft EII/L-150724-3
Certifier JAHANGIR is assigned to Aircraft EIT/L-150724-2
Ce

In [78]:
import pandas as pd
from datetime import datetime, timedelta, time
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value, PULP_CBC_CMD

file_path = r"C:\Users\Alvaro\Documents\Facultad\MBZUAI\Internship\Etihad\Internship\Zonal Allocation\Automated allocations\Automated Allocation Excel v6 - corrected dates.xlsm"
# Load data
manpower_df = pd.read_excel(file_path, sheet_name="Manpower").head(30)
flights_df = pd.read_excel(file_path, sheet_name="Flights").head(30)

# Let's handle the NaN values
manpower_df["Main Certifications"] = manpower_df["Main Certifications"].apply(
    lambda x: [] if pd.isna(x) else x
)
manpower_df["Cat-A Certifications"] = manpower_df["Cat-A Certifications"].apply(
    lambda x: [] if pd.isna(x) else x
)
# Extract certifiers, aircraft, bays, and shifts from the data
certifiers = manpower_df["Name"].unique()
aircraft_wp = flights_df["WP"].unique()
bays = flights_df["Bay"].unique()

# Create dictionaries for certifier and aircraft information
certifier_info = manpower_df.set_index("Name").to_dict("index")
aircraft_info = flights_df.set_index("WP").to_dict("index")

# Define what a long stop is, in minutes
long_stop = 5 * 60


def calculate_movement_cost(bay1, bay2):
    if bay1 // 100 == bay2 // 100:
        return min(2 * abs(bay1 - bay2), 10)
    else:
        return 30


# Create a movement time matrix. This matrix will be for all the possible pairs of aircraft
movement_cost_matrix = {}
for a1 in aircraft_wp:
    for a2 in aircraft_wp:
        movement_cost_matrix[(a1, a2)] = calculate_movement_cost(
            aircraft_info[a1]["Bay"], aircraft_info[a2]["Bay"]
        )


# We can also define a cost of changing bay zones.
def calculate_zone_change_cost(bay_zone_1, bay_zone_2):
    if bay_zone_1 != bay_zone_2:
        return 100
    else:
        return 0


# Costs of assigning Main Certifier, Cat-A Certifier and someone with no certifications
# The No Certifications needs to be done in order to always get a solution
def calculate_certification_cost(certifier, aircraft):
    aircraft_type = aircraft_info[aircraft]["Aircraft type"]
    main_certifications = certifier_info[certifier]["Main Certifications"]
    cat_a_certifications = certifier_info[certifier]["Cat-A Certifications"]
    if aircraft_type in main_certifications:
        return 0
    elif aircraft_type in cat_a_certifications:
        return 50
    else:
        return 100000  # Large penalty to discourage assignment


# Cost of assigning a shift leader
def calculate_shift_leader_cost(certifier):
    if certifier_info[certifier]["Type"] == "LE":
        return 100
    else:
        return 0


# Cost of assigning a quality control certifier
# As the QC is also a shift leader (is this so?), his actual cost will be the quality control cost + the shift leader cost
def calculate_quality_control_cost(certifier):
    if certifier_info[certifier]["Primary Bay Zone"] == "QC":
        return 200
    else:
        return 0


# Define the cost for assigning multiple aircraft to the same certifier. It should not grow linearly, but more exponentially
# 1 aicraft -> 50, 2 aircraft -> 150, 3 aircraft -> 300, 4 aircraft -> 500, 5 aircraft -> 750, 6 aircraft -> 1050
def calculate_assignment_cost(num_assigned_aircraft):
    if num_assigned_aircraft == 0:
        return 0
    if num_assigned_aircraft == 1:
        return 50
    elif num_assigned_aircraft == 2:
        return 150
    elif num_assigned_aircraft == 3:
        return 300
    elif num_assigned_aircraft == 4:
        return 500
    elif num_assigned_aircraft == 5:
        return 750
    elif num_assigned_aircraft == 6:
        return 1050
    else:
        return 2000


# Initialize the problem
problem = LpProblem("Aircraft_Assignment", LpMinimize)
total_cost = 0

# Define decision variables
assignment = LpVariable.dicts("Assign", (certifiers, aircraft_wp), 0, 1, cat="Binary")

# Auxiliary variables for movement
movement = LpVariable.dicts(
    "Movement", (certifiers, aircraft_wp, aircraft_wp), 0, 1, cat="Binary"
)

# Calculate total costs
movement_cost_total = lpSum(
    [
        movement_cost_matrix[(a1, a2)] * movement[c][a1][a2]
        + calculate_zone_change_cost(aircraft_info[a1]["Bay"], aircraft_info[a2]["Bay"])
        * movement[c][a1][a2]
        for c in certifiers
        for a1 in aircraft_wp
        for a2 in aircraft_wp
    ]
)


# Calculate assignment costs and add them to the total cost
# Define binary variables for assignment cost function
assignment_costs = {
    c: LpVariable.dicts(f"AssignmentCost_{c}", range(7), 0, 1, cat="Binary")
    for c in certifiers
}

assignment_cost_total = lpSum(
    [
        calculate_assignment_cost(i) * assignment_costs[c][i]
        for c in certifiers
        for i in range(7)
    ]
)

for c in certifiers:
    # Ensure that exactly one cost level is selected
    problem += lpSum([assignment_costs[c][i] for i in range(7)]) == 1

    # Link assignment cost levels to the number of assigned aircraft
    problem += lpSum([assignment[c][a] for a in aircraft_wp]) == lpSum(
        [i * assignment_costs[c][i] for i in range(7)]
    )

# Add the additional cost components to the total cost
certification_shift_quality_cost_total = lpSum(
    [
        assignment[c][a]
        * (
            calculate_certification_cost(c, a)
            + calculate_shift_leader_cost(c)
            + calculate_quality_control_cost(c)
        )
        for c in certifiers
        for a in aircraft_wp
    ]
)

# Add all components to the total cost
total_cost = (
    movement_cost_total + assignment_cost_total + certification_shift_quality_cost_total
)

# Add the total cost to the problem
problem += total_cost, "Minimize_Total_Cost"

# Constraints
# Each aircraft should be assigned to exactly one certifier
for a in aircraft_wp:
    problem += lpSum([assignment[c][a] for c in certifiers]) == 1

# # Certifiers should be assigned to aircraft they have certification, as Main Certification or Cat-A certification. If they don't have it, they can't be assigned
# for c in certifiers:
#     for a in aircraft_wp:
#         if (
#             aircraft_info[a]["Aircraft type"]
#             not in certifier_info[c]["Main Certifications"]
#             and aircraft_info[a]["Aircraft type"]
#             not in certifier_info[c]["Cat-A Certifications"]
#         ):
#             problem += assignment[c][a] == 0

# Add the constraint that certifiers should be assigned to aircraft in their shift range + 1 hour
for c in certifiers:
    for a in aircraft_wp:
        arrival_time = aircraft_info[a]["Arrival time"]
        departure_time = aircraft_info[a]["Departure time"]

        shift_start_buffered = certifier_info[c]["Shift Start"] - timedelta(hours=1)
        shift_end_buffered = certifier_info[c]["Shift End"] + timedelta(hours=1)

        # Check if the aircraft's arrival or departure time falls within the buffered shift range
        if not (
            shift_start_buffered <= arrival_time <= shift_end_buffered
            or shift_start_buffered <= departure_time <= shift_end_buffered
        ):
            problem += assignment[c][a] == 0

# Certifiers should be assigned to a maximum of 2 aircraft with long layovers
for c in certifiers:
    problem += (
        lpSum(
            [
                assignment[c][a]
                for a in aircraft_wp
                if aircraft_info[a]["Ground Time (minutes)"] > long_stop
            ]
        )
        <= 2
    )

# Link movement variables with assignments
for c in certifiers:
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                problem += (
                    movement[c][a1][a2] >= assignment[c][a1] + assignment[c][a2] - 1
                )
                problem += movement[c][a1][a2] <= assignment[c][a1]
                problem += movement[c][a1][a2] <= assignment[c][a2]

# Solve the problem
solver = PULP_CBC_CMD(msg=1, timeLimit=120, presolve="on", strong=2, gapRel=0.05)
problem.solve(solver)

# Check the solution
print(f"Status: {LpStatus[problem.status]}")
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

# Detailed cost breakdown
print("Detailed Cost Breakdown:")
movement_cost_value = value(movement_cost_total)
assignment_cost_value = value(assignment_cost_total)
certification_shift_quality_cost_value = value(certification_shift_quality_cost_total)

print(f"Movement cost: {movement_cost_value}")
print(f"Assignment cost: {assignment_cost_value}")
print(
    f"Certification, shift leader, and quality control cost: {certification_shift_quality_cost_value}"
)

for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

# Print the costs and assignments
no_certification_assignments = []

for c in certifiers:
    num_aircraft_assigned = sum(assignment[c][a].varValue for a in aircraft_wp)
    assignment_cost = calculate_assignment_cost(num_aircraft_assigned)
    print(f"Certifier {c}:")
    print(f"  Number of aircraft assigned: {num_aircraft_assigned}")
    print(f"  Assignment cost: {assignment_cost}")

    movement_cost = 0
    zone_change_cost = 0
    assigned_aircraft = [a for a in aircraft_wp if assignment[c][a].varValue == 1]

    for i in range(len(assigned_aircraft)):
        for j in range(i + 1, len(assigned_aircraft)):
            a1 = assigned_aircraft[i]
            a2 = assigned_aircraft[j]
            movement_cost += movement_cost_matrix[(a1, a2)]
            zone_change_cost += calculate_zone_change_cost(
                aircraft_info[a1]["Bay"], aircraft_info[a2]["Bay"]
            )

    print(f"  Movement cost: {movement_cost}")
    print(f"  Zone change cost: {zone_change_cost}")

    for a in assigned_aircraft:
        certification_cost = calculate_certification_cost(c, a)
        print(f"  Aircraft {a}:")
        print(f"    Certification cost: {certification_cost}")
        if certification_cost == 100000:
            no_certification_assignments.append((c, a))

# Print assignments without certifications
if no_certification_assignments:
    print("Assignments without proper certifications:")
    for certifier, aircraft in no_certification_assignments:
        print(
            f"  Certifier {certifier} is assigned to Aircraft {aircraft} without proper certification"
        )

# Print total cost
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

# Print final assignments
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Optimal
Total cost: 2907.333333333333
Detailed Cost Breakdown:
Movement cost: 107.3333333333333
Assignment cost: 2250.0
Certification, shift leader, and quality control cost: 550.0
Certifier A.OBAID is assigned to Aircraft AEC/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft EIN/L-150724-4
Certifier SANDEEP is assigned to Aircraft AEB/L-150724-2
Certifier AMIN S is assigned to Aircraft EIX/L-150724-2
Certifier AMIN S is assigned to Aircraft EIX/L-150724-3
Certifier ZAHID is assigned to Aircraft EIV/L-150724-2
Certifier ZAHID is assigned to Aircraft EIS/L-150724-2
Certifier ADIL is assigned to Aircraft EIU/L-150724-3
Certifier ADIL is assigned to Aircraft AEG/L-1G0724-2
Certifier MITSIS is assigned to Aircraft EIW/L-150724-2
Certifier MITSIS is assigned to Aircraft EIP/L-150724-3
Certifier NASEERUDDIN is assigned to Aircraft AEI/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EIA/L-150724-3
Certifier OSMAN is assigned to Aircraft EII/L-150724-3
Certifier OSMAN is 

In [80]:
import pandas as pd

# Initialize lists to store the data
data = []

no_certification_assignments = []

for c in certifiers:
    num_aircraft_assigned = sum(assignment[c][a].varValue for a in aircraft_wp)
    assignment_cost = calculate_assignment_cost(num_aircraft_assigned)

    movement_cost = 0
    zone_change_cost = 0
    assigned_aircraft = [a for a in aircraft_wp if assignment[c][a].varValue == 1]

    for i in range(len(assigned_aircraft)):
        for j in range(i + 1, len(assigned_aircraft)):
            a1 = assigned_aircraft[i]
            a2 = assigned_aircraft[j]
            movement_cost += movement_cost_matrix[(a1, a2)]
            zone_change_cost += calculate_zone_change_cost(
                aircraft_info[a1]["Bay"], aircraft_info[a2]["Bay"]
            )

    for a in assigned_aircraft:
        certification_cost = calculate_certification_cost(c, a)
        data.append(
            {
                "Certifier": c,
                "Aircraft": a,
                "Number of Aircraft Assigned": num_aircraft_assigned,
                "Assignment Cost": assignment_cost,
                "Movement Cost": movement_cost,
                "Zone Change Cost": zone_change_cost,
                "Certification Cost": certification_cost,
            }
        )
        if certification_cost == 100000:
            no_certification_assignments.append((c, a))

# Print assignments without certifications
if no_certification_assignments:
    print("Assignments without proper certifications:")
    for certifier, aircraft in no_certification_assignments:
        print(
            f"  Certifier {certifier} is assigned to Aircraft {aircraft} without proper certification"
        )

# Print total cost
total_cost_value = value(problem.objective)
print(f"Total cost: {total_cost_value}")

# Create a DataFrame and write to Excel
df = pd.DataFrame(data)
df.to_excel("assignment_costs.xlsx", index=False)

print("All costs and assignments have been saved to 'assignment_costs.xlsx'.")

# Print final assignments
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Total cost: 2907.333333333333
All costs and assignments have been saved to 'assignment_costs.xlsx'.
Certifier A.OBAID is assigned to Aircraft AEC/L-150724-2
Certifier A.SHUAIBI is assigned to Aircraft EIN/L-150724-4
Certifier SANDEEP is assigned to Aircraft AEB/L-150724-2
Certifier AMIN S is assigned to Aircraft EIX/L-150724-2
Certifier AMIN S is assigned to Aircraft EIX/L-150724-3
Certifier ZAHID is assigned to Aircraft EIV/L-150724-2
Certifier ZAHID is assigned to Aircraft EIS/L-150724-2
Certifier ADIL is assigned to Aircraft EIU/L-150724-3
Certifier ADIL is assigned to Aircraft AEG/L-1G0724-2
Certifier MITSIS is assigned to Aircraft EIW/L-150724-2
Certifier MITSIS is assigned to Aircraft EIP/L-150724-3
Certifier NASEERUDDIN is assigned to Aircraft AEI/L-150724-2
Certifier NASEERUDDIN is assigned to Aircraft EIA/L-150724-3
Certifier OSMAN is assigned to Aircraft EII/L-150724-3
Certifier OSMAN is assigned to Aircraft EJA/L-150724-3
Certifier JAHANGIR is assigned to Aircraft AEF/L-1507